In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
#Data parameters
image_size = 28*28
n_class = 10

#Trainin parameters
learning_rate = 0.1
total_steps = 1000
batch_size = 100

X = tf.placeholder(tf.float32, shape=[None, image_size], name='InputImage')
Y = tf.placeholder(tf.float32, shape=[None, n_class], name='Label')

In [3]:
'''
def neural_net(self, shape, scope):
    with tf.name_scope(scope):
        W = tf.Variable(tf.random_normal(shape, dtype=tf.float32), name='weight')
        b = tf.Variable(tf.random_normal([shape[1]]), name='bias')
        self = tf.add(tf.matmul(self, W), b)
    return self
'''
def neural_net(self, nodes, scope):
    with tf.name_scope(scope):
        self = tf.layers.dense(self, nodes)
    return self

In [4]:
with tf.name_scope('Model'):
    dense_net = neural_net(X, 128,'layer_1')
    dense_net = neural_net(dense_net, 256,'layer_2')
    dense_net = neural_net(dense_net, n_class,'layer_3')

In [5]:
with tf.name_scope('Loss'):
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=dense_net, labels=Y))
with tf.name_scope("Adam"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op)
with tf.name_scope('Accuracy'):
    correct_pred =  tf.equal(tf.argmax(dense_net, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
init = tf.global_variables_initializer()
tf.summary.scalar('loss', loss_op)
tf.summary.scalar('accuracy', accuracy)
merge_summary_op = tf.summary.merge_all()

In [7]:
with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter("/tmp/tf_tutorial/05_DenseNeuralNetwork", graph=tf.get_default_graph())
    for step in range(total_steps):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        summary, acc, _ = sess.run([merge_summary_op, accuracy, train_op], feed_dict={X:batch_x, Y:batch_y})
        train_writer.add_summary(summary, step)
        if step%100 == 0:
            print("step: {}  accuracy:{:.2f}%".format(step, acc*100))
    print('Training Complete!')
    train_acc = sess.run(accuracy, feed_dict={X:mnist.train.images, Y:mnist.train.labels})
    print("Final Training acc:{:.2f}%".format(train_acc*100))
    test_acc = sess.run(accuracy, feed_dict={X:mnist.test.images[:3000], Y:mnist.test.labels[:3000]})
    print("Testing acc       :{:.2f}%".format(test_acc*100))

step: 0  accuracy:6.00%
step: 100  accuracy:86.00%
step: 200  accuracy:83.00%
step: 300  accuracy:78.00%
step: 400  accuracy:88.00%
step: 500  accuracy:90.00%
step: 600  accuracy:89.00%
step: 700  accuracy:87.00%
step: 800  accuracy:88.00%
step: 900  accuracy:93.00%
Training Complete!
Final Training acc:87.97%
Testing acc       :84.40%
